In [2]:
import os
import sys
import csv
import numpy as np
from sklearn import svm
from sklearn.metrics import confusion_matrix as cmat
import collections
import itertools
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report as report  
from sklearn.model_selection import *
from sklearn.feature_selection import *
import matplotlib.pyplot as plt
import time
import random
import numpy
from datetime import datetime
import pandas as pd
import requests
import re
import xarray as xr
import pickle

In [4]:
directory = '../data/line_p_data/Rosette'
filename = directory+'/'+'cruise-2007-01-station-ros.csv'
ross_data = pd.DataFrame()

ross_data = pd.read_csv(filename)
ross_data

,File Name,Zone,FIL:START TIME YYYY/MM/DD,HH:MM,LOC:EVENT_NUMBER,latitude,longitude,station,Sample_Number,Bottle_Number,...,Flag:Silicate:Bottle,Chlorophyll:Extracted:Bottle [mg/m^3],Flag:Chlorophyll:Extracted:Bottle,Dimethyl_Sulphide:Bottle [nmol/l],Flag:Dimethyl_Sulphide:Bottle,Carbon:Dissolved:Inorganic:Bottle [µmol/kg],Flag:Carbon:Dissolved:Inorganic:Bottle,Alkalinity:Total:Bottle [µmol/kg],Flag:Alkalinity:Total:Bottle,Comments
0,2007-01-0003.che,UTC,09/02/2007,10:21,3,48.57583,-125.49966,1,36,1,...,0.0,0.76,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2007-01-0004.che,UTC,09/02/2007,12:45,4,48.60050,-126.00050,2,44,8,...,0.0,0.87,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2007-01-0006.che,UTC,09/02/2007,14:56,6,48.62533,-126.33334,3,45,1,...,0.0,0.75,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2007-01-0007.che,UTC,09/02/2007,16:48,7,48.65033,-126.66700,4,65,20,...,0.0,NaN,NaN,NaN,NaN,2021.33,0.0,2164.9,0.0,NaN
4,2007-01-0009.che,UTC,09/02/2007,18:48,9,48.65000,-126.66650,4,77,12,...,0.0,0.68,0.0,0.20,0.0,NaN,NaN,NaN,NaN,NaN
5,2007-01-0009.che,UTC,09/02/2007,18:48,9,48.65000,-126.66650,4,78,13,...,0.0,0.63,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
6,2007-01-0009.che,UTC,09/02/2007,18:48,9,48.65000,-126.66650,4,79,14,...,0.0,0.67,0.0,0.22,0.0,NaN,NaN,NaN,NaN,NaN
7,2007-01-0012.che,UTC,09/02/2007,23:00,12,48.69100,-127.16534,5,80,1,...,0.0,0.61,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2007-01-0014.che,UTC,10/02/2007,02:35,14,48.74367,-127.66666,6,81,1,...,0.0,0.59,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2007-01-0017.che,UTC,10/02/2007,06:24,17,48.77717,-128.17084,7,82,1,...,0.0,0.72,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:

for filename in os.listdir(directory):
    each_data = pd.read_csv(directory+'/'+filename, error_bad_lines=False)
    ross_data = pd.concat([ross_data,each_data],axis=0)

ross_data.head(100)

C:\Users\valera\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,HH:MM,Alkalinity:Total:Bottle [µmol/kg],Bottle:Firing_Sequence,Bottle_Number,Carbon:Dissolved:Inorganic:Bottle [µmol/kg],Chlorophyll:Extracted:Bottle,Chlorophyll:Extracted:Bottle [mg/m^3],Comments,Comments_2,Dimethyl_Sulphide:Bottle,...,Unnamed: 42,Zone,latitude,longitude,mass_concentration_of_chlorophyll,moles_of_oxygen_per_unit_mass_in_sea_water,pH:Bottle,sea_water_pressure,sea_water_temperature,station
0,10:21,NaN,NaN,1,NaN,NaN,0.76,NaN,NaN,NaN,...,NaN,UTC,48.57583,-125.49966,NaN,281.6,NaN,NaN,NaN,1
1,12:45,NaN,NaN,8,NaN,NaN,0.87,NaN,NaN,NaN,...,NaN,UTC,48.60050,-126.00050,NaN,280.4,NaN,NaN,NaN,2
2,14:56,NaN,NaN,1,NaN,NaN,0.75,NaN,NaN,NaN,...,NaN,UTC,48.62533,-126.33334,NaN,287.0,NaN,NaN,NaN,3
3,16:48,2164.9,NaN,20,2021.33,NaN,NaN,NaN,NaN,NaN,...,NaN,UTC,48.65033,-126.66700,NaN,286.5,NaN,NaN,NaN,4
4,18:48,NaN,NaN,12,NaN,NaN,0.68,NaN,NaN,NaN,...,NaN,UTC,48.65000,-126.66650,NaN,288.6,NaN,NaN,NaN,4
5,18:48,NaN,NaN,13,NaN,NaN,0.63,NaN,NaN,NaN,...,NaN,UTC,48.65000,-126.66650,NaN,288.6,NaN,NaN,NaN,4
6,18:48,NaN,NaN,14,NaN,NaN,0.67,NaN,NaN,NaN,...,NaN,UTC,48.65000,-126.66650,NaN,288.6,NaN,NaN,NaN,4
7,23:00,NaN,NaN,1,NaN,NaN,0.61,NaN,NaN,NaN,...,NaN,UTC,48.69100,-127.16534,NaN,290.8,NaN,NaN,NaN,5
8,02:35,NaN,NaN,1,NaN,NaN,0.59,NaN,NaN,NaN,...,NaN,UTC,48.74367,-127.66666,NaN,291.6,NaN,NaN,NaN,6
9,06:24,NaN,NaN,1,NaN,NaN,0.72,NaN,NaN,NaN,...,NaN,UTC,48.77717,-128.17084,NaN,293.0,NaN,NaN,NaN,7


In [6]:
ross_data = ross_data.fillna(-999.0)
ross_data.astype(bool).sum(axis=0)

 HH:MM                                                  18878
Alkalinity:Total:Bottle [µmol/kg]                       18878
Bottle:Firing_Sequence                                  18878
Bottle_Number                                           18878
Carbon:Dissolved:Inorganic:Bottle [µmol/kg]             18878
Chlorophyll:Extracted:Bottle                            18878
Chlorophyll:Extracted:Bottle [mg/m^3]                   18878
Comments                                                18878
Comments_2                                              18878
Dimethyl_Sulphide:Bottle                                18868
Dimethyl_Sulphide:Bottle [nmol/l]                       18385
Dimethylsulfoniopropionate_Dissolved:Bottle [nmol/l]    18868
Dimethylsulfoniopropionate_Total:Bottle [nmol/l]        18875
FIL:START TIME DD/MM/YYYY                               18878
FIL:START TIME YYYY/MM/DD                               18878
File Name                                               18878
Flag:Alk

In [15]:
multioutput = ross_data[['moles_of_oxygen_per_unit_mass_in_sea_water','Silicate:Bottle [µmol/l]','Nitrate_plus_Nitrite:Bottle','Phosphate:Bottle [µmol/l]']]

In [19]:
multioutput.fillna(0)
multioutput.astype(bool).sum(axis=0)

moles_of_oxygen_per_unit_mass_in_sea_water    18878
Silicate:Bottle [µmol/l]                      18878
Nitrate_plus_Nitrite:Bottle                   18865
Phosphate:Bottle [µmol/l]                     18878
dtype: int64

In [11]:
parameters = ross_data[['FIL:START TIME YYYY/MM/DD',' HH:MM','latitude','longitude','Pressure:CTD [dbar]','Temperature:CTD [deg_C_(ITS90)]','Salinity:Practical:CTD [PSS-78]']]

In [12]:
parameters

,FIL:START TIME YYYY/MM/DD,HH:MM,latitude,longitude,Pressure:CTD [dbar],Temperature:CTD [deg_C_(ITS90)],Salinity:Practical:CTD [PSS-78]
0,09/02/2007,10:21,48.57583,-125.49966,6.0,8.4299,NaN
1,09/02/2007,12:45,48.60050,-126.00050,5.8,8.2880,NaN
2,09/02/2007,14:56,48.62533,-126.33334,4.6,8.3349,NaN
3,09/02/2007,16:48,48.65033,-126.66700,5.2,8.7896,NaN
4,09/02/2007,18:48,48.65000,-126.66650,7.8,8.7918,NaN
5,09/02/2007,18:48,48.65000,-126.66650,7.7,8.7917,NaN
6,09/02/2007,18:48,48.65000,-126.66650,2.2,8.7938,NaN
7,09/02/2007,23:00,48.69100,-127.16534,4.9,8.4059,NaN
8,10/02/2007,02:35,48.74367,-127.66666,7.2,8.3560,NaN
9,10/02/2007,06:24,48.77717,-128.17084,2.3,8.2108,NaN


In [60]:
ross_data.to_csv(directory+'/'+'ross_data_consolidated.csv',index=False)

In [7]:
ross_data = ross_data.replace(-999,np.nan)
ross_data = ross_data.replace('-999.0',np.nan,regex=True)
ross_data = ross_data.replace('-999.00',np.nan)
ross_data = ross_data.replace('-999.0000',np.nan)

In [8]:
ross_data

,HH:MM,Alkalinity:Total:Bottle [µmol/kg],Bottle:Firing_Sequence,Bottle_Number,Carbon:Dissolved:Inorganic:Bottle [µmol/kg],Chlorophyll:Extracted:Bottle,Chlorophyll:Extracted:Bottle [mg/m^3],Comments,Comments_2,Dimethyl_Sulphide:Bottle,...,Unnamed: 42,Zone,latitude,longitude,mass_concentration_of_chlorophyll,moles_of_oxygen_per_unit_mass_in_sea_water,pH:Bottle,sea_water_pressure,sea_water_temperature,station
0,10:21,NaN,NaN,1,NaN,NaN,0.76,NaN,NaN,NaN,...,NaN,UTC,48.57583,-125.49966,NaN,281.6,NaN,NaN,NaN,1
1,12:45,NaN,NaN,8,NaN,NaN,0.87,NaN,NaN,NaN,...,NaN,UTC,48.60050,-126.00050,NaN,280.4,NaN,NaN,NaN,2
2,14:56,NaN,NaN,1,NaN,NaN,0.75,NaN,NaN,NaN,...,NaN,UTC,48.62533,-126.33334,NaN,287.0,NaN,NaN,NaN,3
3,16:48,2164.9,NaN,20,2021.33,NaN,NaN,NaN,NaN,NaN,...,NaN,UTC,48.65033,-126.66700,NaN,286.5,NaN,NaN,NaN,4
4,18:48,NaN,NaN,12,NaN,NaN,0.68,NaN,NaN,NaN,...,NaN,UTC,48.65000,-126.66650,NaN,288.6,NaN,NaN,NaN,4
5,18:48,NaN,NaN,13,NaN,NaN,0.63,NaN,NaN,NaN,...,NaN,UTC,48.65000,-126.66650,NaN,288.6,NaN,NaN,NaN,4
6,18:48,NaN,NaN,14,NaN,NaN,0.67,NaN,NaN,NaN,...,NaN,UTC,48.65000,-126.66650,NaN,288.6,NaN,NaN,NaN,4
7,23:00,NaN,NaN,1,NaN,NaN,0.61,NaN,NaN,NaN,...,NaN,UTC,48.69100,-127.16534,NaN,290.8,NaN,NaN,NaN,5
8,02:35,NaN,NaN,1,NaN,NaN,0.59,NaN,NaN,NaN,...,NaN,UTC,48.74367,-127.66666,NaN,291.6,NaN,NaN,NaN,6
9,06:24,NaN,NaN,1,NaN,NaN,0.72,NaN,NaN,NaN,...,NaN,UTC,48.77717,-128.17084,NaN,293.0,NaN,NaN,NaN,7
